In [1]:
import preprocess

## 1. 표제어 사전 만들기 

In [17]:
execfile("preprocess.py")
lemm_parser=ParseLemmJson(r'법률 지식베이스\법령용어\법령용어(Json)\nia_law_term_ontology_사전_20180117.json')
lemm_list,uris=lemm_parser.open()
lemm_pd=lemm_parser.toPandas()
lemm_pd.head(1)

In [ ]:
lemm_pd.head(10)

,vocab
0,납세대상자
1,촬영후


## 2. 토큰화, 위치 라벨링 진행하기

### (1) Connect with API

In [2]:

execfile("preprocess.py")
dataParser=ParsePanrye()
soup=dataParser.connect()


              class is initialized, 
              be sure your ip is authenticated by the api
              your ip: 192.168.56.1


In [11]:
soup.preprocess()

'prec*evtNm830841221731강도·폭행·업무방해·부착명령2021도171312022.05.19대법원형사400102전원합의체 판결선고/DRF/lawService.do?OC=mugudeli601&target=prec&ID=221731&type=HTML&mobileYn=221733채무불이행자명부등재2021마63712022.05.17대법원민사400101결정자/DRF/lawService.do?OC=mugudeli601&target=prec&ID=221733&type=HTML&mobileYn=221743물품대금2018다2247812022.05.13대법원민사400101판결선고/DRF/lawService.do?OC=mugudeli601&target=prec&ID=221743&type=HTML&mobileYn=221739재산세부과처분취소2018두501472022.05.13대법원세무400108판결선고/DRF/lawService.do?OC=mugudeli601&target=prec&ID=221739&type=HTML&mobileYn=221741회계장부와서류의열람및등사청구의소2019다2701632022.05.13대법원민사400101판결선고/DRF/lawService.do?OC=mugudeli601&target=prec&ID=221741&type=HTML&mobileYn=221737무고·사문서위조·위조사문서행사2017도38842022.05.13대법원형사400102판결선고/DRF/lawService.do?OC=mugudeli601&target=prec&ID=221737&type=HTML&mobileYn=221747손해배상(기)2019다2157912022.05.13대법원민사400101판결선고/DRF/lawService.do?OC=mugudeli601&target=prec&ID=221747&type=HTML&mobileYn=221735명예훼손2020도156422022.05.13대법원형사400102판결선고/DRF/lawService.do?OC=mugudeli601&target=p

### (2) 원하는 데이터만 추출해보기 

In [3]:
prec=[t.preprocess() for t in soup.find_all("prec")]
for p in prec:
    print(p,"\n")

221945친양자입양신청2020스5142022.05.31대법원가사400103결정자/DRF/lawService.do?OC=mugudeli601&target=prec&ID=221945&type=HTML&mobileYn= 

221947소송비용액확정2022마51412022.05.31대법원민사400101결정자/DRF/lawService.do?OC=mugudeli601&target=prec&ID=221947&type=HTML&mobileYn= 

221973강제추행2017도115822022.05.26대법원형사400102판결선고/DRF/lawService.do?OC=mugudeli601&target=prec&ID=221973&type=HTML&mobileYn= 

221993토지인도등·손해배상금2021다2164212022.05.26대법원민사400101판결선고/DRF/lawService.do?OC=mugudeli601&target=prec&ID=221993&type=HTML&mobileYn= 

221979구상금2022다2114162022.05.26대법원민사400101판결선고/DRF/lawService.do?OC=mugudeli601&target=prec&ID=221979&type=HTML&mobileYn= 

221957특정경제범죄가중처벌등에관한법률위반(배임)2021도24882022.05.26대법원형사400102판결선고/DRF/lawService.do?OC=mugudeli601&target=prec&ID=221957&type=HTML&mobileYn= 

221731강도·폭행·업무방해·부착명령2021도171312022.05.19대법원형사400102전원합의체 판결선고/DRF/lawService.do?OC=mugudeli601&target=prec&ID=221731&type=HTML&mobileYn= 

221733채무불이행자명부등재2021마63712022.05.17대법원민사400101결정자/DRF/lawService.do?OC=mugudeli601&target=prec&I

### 중간 체크업 

*상황* 
현재 target=prec으로 접근할 수 있는 판례 개수는 20개에 지나지 않으며, 아래와 같은 키의 구성으로 되어있다.

- 판례일련번호
- 사건명 (코드명일 때도 있고, ![[CDATA]]형일 때도 있음)
- 사건번호
- 선고일자 (사건번호와 선고일자는 (사건번호,신고일자) 튜플으로 판례/튜플 형식으로 판례일련번호를 대체할 수 있다.)
- 법원명
- 법원종류코드
- 사건종류명
- 사건종류코드
- 판결유형
- 선고
- 판례상세링크 

판례상세링크, 혹은 law.go.kr/판례/(사건번호,선고일자)로 판례 상세를 검색할 수 있다. 

판례 상세페이지는 제목과 요약, 본문으로 구성되어있는데 본문에는 수많은 다른 판례가 참조되어있다. 


*접근방법*<br>

#### 과제 1: 데이터 정제하기 
모든 판례에 대해서 db를 생성하는 과정으로 offline이다. 최대한 많은 판례를 처리하는 것이 목적이다.
처리 이후에는 수집한 데이터를 가지고 클러스터링 내지는 동질 그룹을 파악해 링크 또는 추천을 하는 것이다. 

- 판례가 api로 되는 것만 일단 부른다 (20개) 
- 먼저 판례 요약, 판례 본문, 제목을 literal로 불러온다.
- 판례 링크가 첨부되어있는 줄은 중요한 줄로 따로 위치와 링크를 저장해둔다. 
- 판례는 전부 [PANRYE]로 토큰화하고, 나머지는 일반적으로 토큰화한다.
- 표제어 사전을 거쳐 불필요한 토큰 분절을 방지한다. 

#### 과제 2: 답안지가 필요 없는 textRank로 중요 단어, 중요 문장 추출 시도하기
- TextRank를 연결한다

#### 과제 3: 판례문이 붙어있는 문장만 모아놓기
#### 과제 4: 추천시스템의 답을 찾기
가장 이상적인 것은 bert나 gpt-3로 중요 단어, 문장의 순위를 구하는 것인데 우리가 답안지가 없다. 

지금 판례들이 서로 링크가 되어있는데<br>
X판례 내용-->Y스스로가 링크된 판례를 가리키는 one hot vector 느낌의 답안지<br>
를 구성해서 딥러닝 모델의 판례 embedding을 구하고 나면<br>
클러스터링이 가능할 것으로 보인다. <br>

아니면 
X판례 내용 --> Y textRank 결과 단어들으로<br>
딥러닝 모델에 넣어 돌려 판례 embedding을 구하고 나면<br>
그걸로 클러스터링을 해도 괜찮을 것 같다.<br>

클러스터링의 유효성 판단은... 사용자 확인 외에 뭐가 또 있을까? 

### (3) literal 하게 본문 긁어오기 

In [14]:
execfile("preprocess.py")
dataParser=ParsePanrye()
match,yoji=dataParser.parsePrecDetail()


              class is initialized, 
              be sure your ip is authenticated by the api
              your ip: 192.168.56.1
221945 
 3 

221947 
 4 

221961 
 4 

221997 
 4 

221951 
 4 

221953 
 4 

221985 
 4 

221971 
 4 

221983 
 4 

221963 
 3 

221977 
 3 

221979 
 3 

221959 
 4 

221965 
 3 

221967 
 3 

221989 
 4 

221975 
 4 

221991 
 3 

221955 
 3 

221957 
 4 



({'221945': 'https://www.law.go.kr/LSW/precInfoP.do?precSeq=221945&mode=0',
  '221947': 'https://www.law.go.kr/LSW/precInfoP.do?precSeq=221947&mode=0',
  '221961': 'https://www.law.go.kr/LSW/precInfoP.do?precSeq=221961&mode=0',
  '221997': 'https://www.law.go.kr/LSW/precInfoP.do?precSeq=221997&mode=0',
  '221951': 'https://www.law.go.kr/LSW/precInfoP.do?precSeq=221951&mode=0',
  '221953': 'https://www.law.go.kr/LSW/precInfoP.do?precSeq=221953&mode=0',
  '221985': 'https://www.law.go.kr/LSW/precInfoP.do?precSeq=221985&mode=0',
  '221971': 'https://www.law.go.kr/LSW/precInfoP.do?precSeq=221971&mode=0',
  '221983': 'https://www.law.go.kr/LSW/precInfoP.do?precSeq=221983&mode=0',
  '221963': 'https://www.law.go.kr/LSW/precInfoP.do?precSeq=221963&mode=0',
  '221977': 'https://www.law.go.kr/LSW/precInfoP.do?precSeq=221977&mode=0',
  '221979': 'https://www.law.go.kr/LSW/precInfoP.do?precSeq=221979&mode=0',
  '221959': 'https://www.law.go.kr/LSW/precInfoP.do?precSeq=221959&mode=0',
  '221965': 

###  (4) 토큰화하기

## 3. 빈칸-답안 쌍 오브젝트 만들기 